In [1]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [3]:
hook_injection = win.extract_nested_fields("hook_injection_2020-01-13170701.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 7..
[+] DataFrame Returned !


In [4]:
hook_injection.createOrReplaceTempView("hook_injection")

In [5]:
Hook_Injection = spark.sql(
'''
SELECT 
    b.computer_name, 
    b.User,
    b.ParentImage,
    d.TargetImage,
    b.LogonId,
    ltrim('0x', d.GrantedAccess) as granted_access
FROM hook_injection b  
JOIN hook_injection d
    ON b.ProcessGuid = d.TargetProcessGUID
    AND d.channel = "Microsoft-Windows-Sysmon/Operational"
    AND d.event_id = 10
    AND b.ParentProcessGuid = d.SourceProcessGUID
    AND b.Image = d.TargetImage
JOIN hook_injection e
    ON e.ProcessGuid = b.ParentProcessGuid
    AND e.channel = "Microsoft-Windows-Sysmon/Operational"
    AND e.event_id = 5
    AND e.Image = d.SourceImage
    AND b.ParentImage = e.Image
JOIN hook_injection f
    ON f.ProcessGuid = b.ParentProcessGuid
    AND f.channel = "Microsoft-Windows-Sysmon/Operational"
    AND f.event_id = 7
    AND f.Image = b.ParentImage
    AND f.Image = f.ImageLoaded
WHERE b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 1
'''
).show(1,False)

+------------------+-----------+--------------------------------------+-------------------------------+-------+--------------+
|computer_name     |User       |ParentImage                           |TargetImage                    |LogonId|granted_access|
+------------------+-----------+--------------------------------------+-------------------------------+-------+--------------+
|win10.marvel.local|MARVEL\thor|C:\Users\thor\Downloads\InjectProc.exe|C:\Windows\System32\notepad.exe|0xdcff8|1fffff        |
+------------------+-----------+--------------------------------------+-------------------------------+-------+--------------+

